In [1]:
# change directory
import os
rootdir = os.path.dirname(os.getcwd())
os.chdir(rootdir)

# import third libs for data
import datetime as dt
import numpy as np
import pandas as pd

# import config
from config import DATA_DIR

# Chargement des donnees

In [2]:
benchmarkname = 'HGD_dialysis_2018_2021'
benchmarkdir = os.path.join(DATA_DIR, 'raw', benchmarkname)
benchmarkdir

'D:\\Recherches\\Travaux\\RCEMP\\rcemp-dial\\data\\raw\\HGD_dialysis_2018_2021'

## chargement des seances

In [3]:
datapath = os.path.join(benchmarkdir, 'donnees_seances.csv')
seances = pd.read_csv(datapath, sep=';', index_col='Date', parse_dates=True, dayfirst=True)
seances.head()

,ID,Tour,Patients,Debut,Duree
Date,,,,,
2018-03-26,1,Tour1,21,5,5
2018-03-26,2,Tour2,21,10,5
2018-03-26,3,Tour3,21,15,5
2018-03-26,4,Tour4,8,20,5
2018-03-27,5,Tour1,21,5,5


In [4]:
seances.dtypes

ID           int64
Tour        object
Patients     int64
Debut        int64
Duree        int64
dtype: object

## chargement des generateurs

In [5]:
datapath = os.path.join(benchmarkdir, 'donnees_generateurs.csv')
generateurs = pd.read_csv(datapath, sep=';', index_col='Numero')
generateurs.head()

,Code,TT,TO,TNR,TR,TAP,TF,TE,TN,TU,TRE,TRSi
Numero,,,,,,,,,,,,
1,7SXAWU98,30576,22932,932,22000,6301.08,15698.92,4.92,15694,15694,0,NaN
2,7SXAWT97,30576,22932,932,22000,19262.22,2737.78,1.78,2736,2736,0,NaN
3,7SXAWU81,30576,22932,932,22000,9462.73,12537.27,1.27,12536,12536,0,NaN
4,7SXAWU91,30576,22932,932,22000,7857.71,14142.29,2.29,14140,14140,0,NaN
5,7SXAWS81,30576,22932,932,22000,5639.52,16360.48,4.48,16356,16356,0,NaN


In [6]:
generateurs.dtypes

Code     object
TT        int64
TO        int64
TNR       int64
TR        int64
TAP     float64
TF      float64
TE      float64
TN        int64
TU        int64
TRE       int64
TRSi    float64
dtype: object

In [7]:
generateurs.Code.sort_values()

Numero
24      6SX4LV10
21      7SXAWS80
5       7SXAWS81
20      7SXAWS82
2       7SXAWT97
8       7SXAWT98
14      7SXAWU79
17      7SXAWU80
3       7SXAWU81
12      7SXAWU82
22      7SXAWU83
18      7SXAWU84
19    7SXAWU85-A
23    7SXAWU85-B
9       7SXAWU86
7       7SXAWU87
11      7SXAWU89
16      7SXAWU90
4       7SXAWU91
15      7SXAWU93
13      7SXAWU94
6       7SXAWU96
10      7SXAWU97
1       7SXAWU98
Name: Code, dtype: object

# Identification des patients

Constante utilisee pour l'identification des patients:

- $\delta_1$  duree du cycle de planification
- $\delta_2$  duree du repos entre traitement
- $\alpha_1$  le nombre de seance par cycle pour un patient 

Notons:

- $P$ le nombre de patients par cycle
- $S$ le nombre de seance effective par cycle

On a :   $ \alpha_1 P = S $

**NB: il existe des cas exceptionnel et marginal ou le nombre de seances est augmente par le medecin au vu de l'etat d'un patient. On considere dans cette exemple que ces cas sont marginaux. Il existe aussi des patients non enregistree qui viennent pour une dialyse de passage**

In [8]:
delta_1 = 1
delta_2 = 24
alpha_1 = 2
sample_percent = 0.14

## calcul des preferences des patients

In [9]:
seances_copy = seances.reset_index()
seances_copy['Preference'] = seances_copy['Tour']
seances_copy['Preference'].replace({'Tour4':'Tour3'}, inplace=True)
seances_copy.head()

,Date,ID,Tour,Patients,Debut,Duree,Preference
0,2018-03-26,1,Tour1,21,5,5,Tour1
1,2018-03-26,2,Tour2,21,10,5,Tour2
2,2018-03-26,3,Tour3,21,15,5,Tour3
3,2018-03-26,4,Tour4,8,20,5,Tour3
4,2018-03-27,5,Tour1,21,5,5,Tour1


In [10]:
seances_group = seances_copy.groupby(['Date', 'Preference'])
seances_sum = seances_group.sum(numeric_only=True)
seances_sum.head()

ID  Patients  Debut  Duree
Date       Preference                            
2018-03-26 Tour1        1        21      5      5
           Tour2        2        21     10      5
           Tour3        7        29     35     10
2018-03-27 Tour1        5        21      5      5
           Tour2        6        21     10      5

In [11]:
seances_min = seances_group.min()
seances_min.head()

ID   Tour  Patients  Debut  Duree
Date       Preference                                   
2018-03-26 Tour1        1  Tour1        21      5      5
           Tour2        2  Tour2        21     10      5
           Tour3        3  Tour3         8     15      5
2018-03-27 Tour1        5  Tour1        21      5      5
           Tour2        6  Tour2        21     10      5

In [12]:
patients = seances_sum['Patients']
patients = pd.concat([patients, seances_min[['Debut', 'Duree']]], axis=1)
patients.reset_index(inplace=True)
# patients.set_index('Date', inplace=True)
patients.head(10)

,Date,Preference,Patients,Debut,Duree
0,2018-03-26,Tour1,21,5,5
1,2018-03-26,Tour2,21,10,5
2,2018-03-26,Tour3,29,15,5
3,2018-03-27,Tour1,21,5,5
4,2018-03-27,Tour2,21,10,5
5,2018-03-27,Tour3,30,15,5
6,2018-03-28,Tour1,21,5,5
7,2018-03-28,Tour2,21,10,5
8,2018-03-28,Tour3,26,15,5
9,2018-03-29,Tour1,21,5,5


## regroupement des patients par cycles, tours et jours

In [13]:
sampling = '%sW' % delta_1
patients['Day'] = patients['Date'].dt.weekday + 1
patients['period'] = patients['Date'].dt.to_period(sampling).apply(lambda r: r.end_time.date)
patients.head(25)

,Date,Preference,Patients,Debut,Duree,Day,period
0,2018-03-26,Tour1,21,5,5,1,2018-04-01
1,2018-03-26,Tour2,21,10,5,1,2018-04-01
2,2018-03-26,Tour3,29,15,5,1,2018-04-01
3,2018-03-27,Tour1,21,5,5,2,2018-04-01
4,2018-03-27,Tour2,21,10,5,2,2018-04-01
5,2018-03-27,Tour3,30,15,5,2,2018-04-01
6,2018-03-28,Tour1,21,5,5,3,2018-04-01
7,2018-03-28,Tour2,21,10,5,3,2018-04-01
8,2018-03-28,Tour3,26,15,5,3,2018-04-01
9,2018-03-29,Tour1,21,5,5,4,2018-04-01


## Selection des patients

In [14]:
selection = patients.copy()
selection.columns = [col.lower() for col in selection.columns]
selection.index.name = 'numero'
selection.head()

,date,preference,patients,debut,duree,day,period
numero,,,,,,,
0,2018-03-26,Tour1,21,5,5,1,2018-04-01
1,2018-03-26,Tour2,21,10,5,1,2018-04-01
2,2018-03-26,Tour3,29,15,5,1,2018-04-01
3,2018-03-27,Tour1,21,5,5,2,2018-04-01
4,2018-03-27,Tour2,21,10,5,2,2018-04-01


In [15]:
f = lambda val:int(val*sample_percent)
selection['patients'] = selection['patients'].apply(f)
selection.head()

,date,preference,patients,debut,duree,day,period
numero,,,,,,,
0,2018-03-26,Tour1,2,5,5,1,2018-04-01
1,2018-03-26,Tour2,2,10,5,1,2018-04-01
2,2018-03-26,Tour3,4,15,5,1,2018-04-01
3,2018-03-27,Tour1,2,5,5,2,2018-04-01
4,2018-03-27,Tour2,2,10,5,2,2018-04-01


## Enregistrement des parametres des patients

In [16]:
patientpath = os.path.join(benchmarkdir, 'parametres_patients.csv')
selection.to_csv(patientpath)

In [17]:
# tourpath = os.path.join(benchmarkdir, 'tours.csv')
# tours.to_csv(tourpath)

# Identification des machines

Notons:

- $\delta_3$ duree traitement pour une seance et une machine en heure
- $\delta_4$ duree demarrage journaliere en heure
- $\delta_5$ nombre d'heures avant maintenance systematique des filtres
- $\delta_6$ nombre d'heures avant maintenance systematique generale
- $\alpha_2$ nombre de lits qui indique le nombre de machines utiles


In [18]:
# pour freysinus
delta_3 = 4
delta_4 = 5 * 4
delta_5 = 2 * 7 * 24                # 2 semaines
delta_6 = 365 * 24                  # 1 an
alpha_2 = int(21 * sample_percent)  # 21 lits avec le pourcentage d'echantillon                      

## calcul de la repartition des charges entre machines

### calcul du nombre de seances par cycles

In [19]:
sampling = '%sW' % delta_1
nb_seances = seances['Patients'].resample(sampling).sum()
nb_seances.name = 'seance_effective'
nb_seances.head()

Date
2018-04-01    410
2018-04-08    408
2018-04-15    416
2018-04-22    408
2018-04-29    408
Freq: W-SUN, Name: seance_effective, dtype: int64

### calcul du nombre de patients par cycles

In [20]:
nb_patients = nb_seances // alpha_1
nb_patients.name = 'nb_patient'
nb_patients.head()

Date
2018-04-01    205
2018-04-08    204
2018-04-15    208
2018-04-22    204
2018-04-29    204
Freq: W-SUN, Name: nb_patient, dtype: int64

In [21]:
stat_patients = pd.concat([nb_seances, nb_patients], axis=1)
stat_patients.head()

,seance_effective,nb_patient
Date,,
2018-04-01,410,205
2018-04-08,408,204
2018-04-15,416,208
2018-04-22,408,204
2018-04-29,408,204


In [22]:
nb_seance_effective = stat_patients['seance_effective'].sum()
temps_total_theorique = nb_seance_effective * delta_3
temps_total_theorique

266164

In [23]:
temps_effectif = generateurs['TF']
temps_total_effectif = temps_effectif.sum()
temps_total_effectif

286763.1

In [24]:
repartition = temps_effectif / temps_total_effectif
repartition.head()

Numero
1    0.054745
2    0.009547
3    0.043720
4    0.049317
5    0.057052
Name: TF, dtype: float64

## calcul des temps de fonctionnement par periodes

In [25]:
temps_echantillon = repartition * temps_total_theorique
temps_echantillon.head()

Numero
1    14571.216948
2     2541.116608
3    11636.678263
4    13126.404602
5    15185.255002
Name: TF, dtype: float64

In [26]:
temps_origine = temps_effectif - temps_echantillon
temps_origine.head()

Numero
1    1127.703052
2     196.663392
3     900.591737
4    1015.885398
5    1175.224998
Name: TF, dtype: float64

In [27]:
cumul_seance = stat_patients['seance_effective'].cumsum()
cumul_seance.name = 'seance_cumulee'
cumul_seance

Date
2018-04-01      410
2018-04-08      818
2018-04-15     1234
2018-04-22     1642
2018-04-29     2050
              ...  
2021-04-25    64938
2021-05-02    65342
2021-05-09    65740
2021-05-16    66143
2021-05-23    66541
Freq: W-SUN, Name: seance_cumulee, Length: 165, dtype: int64

## calcul des dates de mise en service

In [28]:
base_t = dt.datetime(2015, 12, 31)
first_t = seances.index[0]
last_t = stat_patients.index[-1]
base_t, first_t, last_t

(datetime.datetime(2015, 12, 31, 0, 0),
 Timestamp('2018-03-26 00:00:00'),
 Timestamp('2021-05-23 00:00:00', freq='W-SUN'))

In [29]:
# hour = pd.tseries.offsets.Hour()
# total_hours = generateurs['TT'].iloc[0]
# base_t = last_t - total_hours * hour
# base_t, total_hours

In [30]:
# periodes = seances['Tour'].groupby(level=0).count()
# periodes

In [31]:
last_t.dayofweek

6

## calcul des durees de vie des machines

In [32]:
jours = pd.date_range(base_t, last_t, freq='D')
jours

DatetimeIndex(['2015-12-31', '2016-01-01', '2016-01-02', '2016-01-03',
               '2016-01-04', '2016-01-05', '2016-01-06', '2016-01-07',
               '2016-01-08', '2016-01-09',
               ...
               '2021-05-14', '2021-05-15', '2021-05-16', '2021-05-17',
               '2021-05-18', '2021-05-19', '2021-05-20', '2021-05-21',
               '2021-05-22', '2021-05-23'],
              dtype='datetime64[ns]', length=1971, freq='D')

In [33]:
duree_jours = pd.Series(24, index=jours)
duree_cumulee = duree_jours.cumsum()
duree_cumulee

2015-12-31       24
2016-01-01       48
2016-01-02       72
2016-01-03       96
2016-01-04      120
              ...  
2021-05-19    47208
2021-05-20    47232
2021-05-21    47256
2021-05-22    47280
2021-05-23    47304
Freq: D, Length: 1971, dtype: int64

In [34]:
# duree_seances = pd.Series(delta_4, index=periodes.index)
# duree_seances

In [35]:
# duree_cycles  = duree_seances.resample(sampling).sum()
# duree_cycles.head()

In [36]:
# duree_echantillon = duree_seances.sum()
# duree_finale = total_hours
# duree_initiale = duree_finale - duree_echantillon
# duree_initiale, duree_finale

In [37]:
# duree_cumulee = duree_cycles.cumsum()
# duree_cumulee += duree_initiale
# duree_cumulee

## creation des parametres de machines

In [38]:
generateurs.columns

Index(['Code', 'TT', 'TO', 'TNR', 'TR', 'TAP', 'TF', 'TE', 'TN', 'TU', 'TRE',
       'TRSi'],
      dtype='object')

In [39]:
machines = []
code_machines = generateurs['Code']
for time in cumul_seance.index:
    seance_machines = repartition * cumul_seance.loc[time]
    temps_machines = temps_origine + delta_3 * seance_machines
    temps_machines.name = 'duree_usage'
    cols = [code_machines, temps_machines]
    machines_period = pd.concat(cols, axis=1)
    machines_period['periode'] = time
    machines_period['duree_vie'] = duree_cumulee.loc[time]
    machines_period['delai_maintenance_filtre'] = delta_5
    machines_period['delai_maintenance_generale'] = delta_6  
    machines_echantillon = machines_period.sample(n=alpha_2, weights=repartition)
    machines.append(machines_echantillon)
machines = pd.concat(machines)
machines
    

,Code,duree_usage,periode,duree_vie,delai_maintenance_filtre,delai_maintenance_generale
Numero,,,,,,
11,7SXAWU89,1175.416327,2018-04-01,19752,336,8760
17,7SXAWU80,1212.681694,2018-04-01,19752,336,8760
14,7SXAWU79,1279.395050,2018-04-08,19920,336,8760
11,7SXAWU89,1261.673389,2018-04-08,19920,336,8760
18,7SXAWU84,1401.727020,2018-04-15,20088,336,8760
...,...,...,...,...,...,...
7,7SXAWU87,16091.013904,2021-05-09,46968,336,8760
3,7SXAWU81,12467.667831,2021-05-16,47136,336,8760
6,7SXAWU96,14095.112437,2021-05-16,47136,336,8760


In [40]:
24 * 365


8760

## verification des calculs de temps de fonctionnement

In [41]:
period_test = cumul_seance.index[-1]
machines_test = machines[machines.periode==period_test]
machines_test = machines_test[['Code', 'duree_usage']]
generateurs_test = generateurs[['Code', 'TF']]
test = pd.merge(machines_test, generateurs_test, on='Code')
test.head()

,Code,duree_usage,TF
0,7SXAWU81,12537.27,12537.27
1,7SXAWU83,16496.11,16496.11


## selection des machines
equivalence machines et nombres de lits

In [42]:
# Weights will be re-normalized automatically
# test_weights = [0.2, 0.2, 0.2, 0.4]
# df1.sample(n=3, weights=test_weights)
generateurs.sample(n=alpha_2, weights=generateurs.TU)

,Code,TT,TO,TNR,TR,TAP,TF,TE,TN,TU,TRE,TRSi
Numero,,,,,,,,,,,,
19,7SXAWU85-A,30576,22932,932,22000,6365.03,15634.97,2.97,15632,15632,0,NaN
16,7SXAWU90,30576,22932,932,22000,5272.07,16727.93,3.93,16724,16724,0,NaN


## enregistrement des parametres de machines

In [43]:
machinepath = os.path.join(benchmarkdir, 'parametres_machines.csv')
machines.to_csv(machinepath)

In [44]:
machines.dtypes

Code                                  object
duree_usage                          float64
periode                       datetime64[ns]
duree_vie                              int64
delai_maintenance_filtre               int64
delai_maintenance_generale             int64
dtype: object

# Identification des tranches horaires

In [45]:
heures = ['%sH-%sH' % (i, i+1) for i in range(24)]
jours = ['D%s' % (j+1) for j in range(7)]
tranches = pd.DataFrame(data='--', index=heures, columns=jours)
tranches.index.name = 'Period'
tranches

,D1,D2,D3,D4,D5,D6,D7
Period,,,,,,,
0H-1H,--,--,--,--,--,--,--
1H-2H,--,--,--,--,--,--,--
2H-3H,--,--,--,--,--,--,--
3H-4H,--,--,--,--,--,--,--
4H-5H,--,--,--,--,--,--,--
5H-6H,--,--,--,--,--,--,--
6H-7H,--,--,--,--,--,--,--
7H-8H,--,--,--,--,--,--,--
8H-9H,--,--,--,--,--,--,--


## calculs des tranches de soins pour patients

In [46]:
tranches_soins = tranches.copy()
tranches_soins.iloc[5:10, :6] = 'soins1'
tranches_soins.iloc[10:15, :6] = 'soins2'
tranches_soins.iloc[15:20, :6] = 'soins3'
tranches_soins.iloc[20:24, :6] = 'soins4'
tranches_soins.iloc[0:1, 1:7] = 'soins4'
tranches_soins

,D1,D2,D3,D4,D5,D6,D7
Period,,,,,,,
0H-1H,--,soins4,soins4,soins4,soins4,soins4,soins4
1H-2H,--,--,--,--,--,--,--
2H-3H,--,--,--,--,--,--,--
3H-4H,--,--,--,--,--,--,--
4H-5H,--,--,--,--,--,--,--
5H-6H,soins1,soins1,soins1,soins1,soins1,soins1,--
6H-7H,soins1,soins1,soins1,soins1,soins1,soins1,--
7H-8H,soins1,soins1,soins1,soins1,soins1,soins1,--
8H-9H,soins1,soins1,soins1,soins1,soins1,soins1,--


## calculs des tranches de maintenances des machines

In [47]:
tranches_maintenances = tranches.copy()
tranches_maintenances.iloc[7:17, :6] = 'service'
tranches_maintenances

,D1,D2,D3,D4,D5,D6,D7
Period,,,,,,,
0H-1H,--,--,--,--,--,--,--
1H-2H,--,--,--,--,--,--,--
2H-3H,--,--,--,--,--,--,--
3H-4H,--,--,--,--,--,--,--
4H-5H,--,--,--,--,--,--,--
5H-6H,--,--,--,--,--,--,--
6H-7H,--,--,--,--,--,--,--
7H-8H,service,service,service,service,service,service,--
8H-9H,service,service,service,service,service,service,--


## enregistrement des tranches horaires

In [48]:
tranchepath = os.path.join(benchmarkdir, 'tranches_soins.csv')
tranches_soins.to_csv(tranchepath)

In [49]:
tranchepath = os.path.join(benchmarkdir, 'tranches_maintenances.csv')
tranches_maintenances.to_csv(tranchepath)